In [ ]:
import os
import glob
import sys
sys.path.append('../../activedrops')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
from ipywidgets import interact, FloatSlider
import matplotlib.gridspec as gridspec
from scipy.ndimage import gaussian_filter
import colorcet as cc
import holoviews as hv
import panel as pn
pn.extension()


import pivdrops
pivdrops.set_plotting_style()

# Analysis of ActiveDROPS PIV data

ActiveDROPS imaging is carried out in two channels corresponding to microtubules (Cy5) and fluorescent beads (YFP). For PIV analysis we take only the beads channel and process the images (stored in .tiff files) with Matlab PIVlab. We choose an interrogation window of 128 pixels (~175.34 um), and step sizes of 64 and 32 pixels (87.67 um and 43.84 um, respectively). This means we will have a grid of 128 x 128 velocity vectors pointing the direction of the flow. The imaging interval varies across samples, but for this first example we took one frame every three seconds.

In this notebook we show the pipeline used to analyze ActiveDROPS data. PIVlab produces a .txt file per frame, which contains our grid of vectors with velocity data, as well as other features like divergence, vorticity, etc. We first need code that takes our .txt file and gives us pandas dataframe to work with.

In [ ]:
link = "../../data/k401bio-250nM-piv/piv_data/PIVlab_****.txt"
files = sorted(glob.glob(link))

df = pd.read_csv(files[0], skiprows=2).fillna(0)

df['x [um]'] = df['x [m]'] * 1E6
df['y [um]'] = df['y [m]'] * 1E6
df['magnitude [um/s]'] = df['magnitude [m/s]'] * 1E6

df

As we can see, from this table we already have a lot of features we can visualize. For example, the velocity magnitude of each component.

In [ ]:
def drop_heatmap(df, feature):

    vals = df.pivot(index='y [um]', columns='x [um]', values=feature).values

    # Plot the magnitude of the velocity
    plt.figure(figsize=(10, 6))  # Set the figure size
    im = plt.imshow(vals, cmap='viridis', origin='lower', extent=[-2762/2, 2762/2, -2762/2, 2762/2], vmin=vals.min(), vmax=vals.max())
    plt.xlabel('x [um]')
    plt.ylabel('y [um]')
    cbar = plt.colorbar(im)
    cbar.set_label(feature)
    plt.title('Velocity magnitude')
    plt.show()

drop_heatmap(df, "vorticity [1/s]")

## Spatial autocorrelation

Before moving forward with plotting more features, we need to know the relevant scales of the flow, which we can later use to calculate the measured energy of the system over time. Let's calculate the spatial autocorrelation of the velocity data to illustrate the lengths at which vectors are correlated. To do so, we need to follow a few steps.

First we need to apply a Fourier Transform to the velocity magnitude of our data, so we convert it from the spatial to the frequency domain. What does that mean? Let's illustrate it with an example:

The Fourier transform is a mathematical operation that transforms a time-domain signal into its frequency-domain representation. The formula includes the term $e^{-i \omega t}$ (where $i$ is the imaginary unit, $ \omega $ is the angular frequency, and $ t $ is time) because it represents a complex exponential function which can be used to model sinusoids.

The use of $ e^{-i \omega t} $ comes from Euler's formula, which states that:

$$
e^{i\theta} = \cos(\theta) + i\sin(\theta)
$$

From which we can derive Euler's identity when the angle is half a circunference ($\pi$):

$$
e^{i\pi} = -1
$$

Euler's formula says that spinning a point around a circle in the complex plane is like a combination of sliding back and forth along two perpendicular lines, one for cosine in the real plane and one for sine in the imaginary plane.

In [ ]:
# Function to plot Euler's formula with a variable radius
def plot_euler(theta, radius):
    # Calculate the point on the circle
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    
    # Set up the figure and axis
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    
    # Draw the unit circle scaled by the radius
    circle = plt.Circle((0, 0), radius, fill=True, alpha=0.2)
    ax.add_artist(circle)
    
    # Plot the real and imaginary parts
    ax.plot([0, x], [0, y], 'r')
    ax.plot(x, y, 'ro')
    
    # Annotations
    complex_number = f'{x:.2f} + {y:.2f}i'
    ax.annotate(complex_number, xy=(x, y), xytext=(x+0.1, y),
                arrowprops=dict(facecolor='black', shrink=0.05))
    
    # Fixed limits for the plot
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_aspect('equal')
    
    # Grid, legend, and titles
    ax.grid(True)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    plt.title("Visualization of Euler's Formula")
    plt.xlabel('Real part')
    plt.ylabel('Imaginary part')
    plt.show()

# Create the sliders
theta_slider = FloatSlider(min=0, max=2*np.pi, step=0.01, value=np.pi, description='Theta')
radius_slider = FloatSlider(min=0.1, max=2, step=0.1, value=1, description='Radius')

# Show the interactive plot
interact(plot_euler, theta=theta_slider, radius=radius_slider);



The general form of the continuous Fourier transform of a time-domain function $ f(t) $ into a frequency-domain function $ F(\omega) $ is:

$$
F(\omega) = \int_{-\infty}^{\infty} f(t) e^{-i \omega t} \, dt
$$

The negative sign in the exponential ($ e^{-i \omega t} $) indicates that the transform is analyzing the signal in terms of sinusoids that rotate in the counterclockwise direction, which is the standard positive direction of rotation in the complex plane. This is a convention that ensures when you apply the inverse Fourier transform, you get back your original time-domain signal.


In this equation:
- $ F(\omega) $ is the Fourier transform of $ f(t) $.
- $ \omega $ is the angular frequency, related to the frequency $ f $ by $ \omega = 2\pi f $.
- $ t $ is time.
- The integral sums (integrates) the function $ f(t) $ multiplied by a rotating complex exponential across all time.

The Fourier transform breaks down $ f(t) $ into its constituent frequencies, and $ F(\omega) $ tells you the amplitude and phase of each of those frequencies. The magnitude $ |F(\omega)| $ gives the amplitude spectrum, and the argument $ \arg(F(\omega)) $ gives the phase spectrum of the function $ f(t) $.


Since we have discrete values, we'll use the Discrete Fourier Transform, but the difference is just computational:

$$
X(k) = \sum_{n=0}^{N-1} x(n) \cdot e^{-i 2 \pi k n / N}
$$

where:
- $ X(k) $ is the output of the DFT at frequency $ k $.
- $ x(n) $ is the input data at point $ n $.
- $ N $ is the total number of points in the data.
- The exponential term $ e^{-i 2 \pi k n / N} $ represents the complex sinusoids (comprising both sine and cosine terms) at frequency $ k $.

The operation is performed for each frequency $ k $ from 0 to $ N-1 $, covering the whole spectrum from the lowest to the highest frequency that the grid can represent.

From our data, we can do fast Fourier transform with numpy and visualize the energy spectrum.

In [ ]:
v = df.pivot(index='y [um]', columns='x [um]', values="magnitude [um/s]").values

v_ft = np.fft.fftshift(np.fft.fft2(v))

# Compute the radial energy spectrum (averaging over circular shells in k-space)
kx = np.fft.fftshift(np.fft.fftfreq(v.shape[0]))
ky = np.fft.fftshift(np.fft.fftfreq(v.shape[1]))
kx, ky = np.meshgrid(kx, ky)
k = np.sqrt(kx**2 + ky**2)

# Define the bins for k (radial wavenumbers)
k_bins = np.linspace(0, 0.5, v.shape[0]//2)
k_bin_centers = 0.5 * (k_bins[:-1] + k_bins[1:])

E_u = np.histogram(k.ravel(), bins=k_bins, weights=(np.abs(v_ft)**2).ravel())[0]

# Plotting the average energy spectrum
plt.figure(figsize=(12, 6))
plt.loglog(k_bin_centers, E_u)
plt.xlabel('Wavenumber')
plt.ylabel('Energy Density')
plt.title('Energy Spectrum')
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()


### Spatial Autocorrelation Function:

The spatial autocorrelation function quantifies how similar the velocities are across different spatial lags in the vector field. If we have a velocity vector field defined as $\mathbf{v}(x,y)$, where $\mathbf{v}$ is the velocity at position $(x,y)$, the spatial autocorrelation function $R(\Delta x, \Delta y)$ for spatial lags $\Delta x$ and $\Delta y$ is given by:

$$
R(\Delta x, \Delta y) = \sum_{x,y} \mathbf{v}(x,y) \cdot \mathbf{v}(x + \Delta x, y + \Delta y)
$$

This function computes the dot product of velocity vectors at the original and shifted positions, thus incorporating both the magnitude and direction of the velocities into the correlation.

### Wiener-Khinchin Theorem for Spatial Data:

The Wiener-Khinchin theorem states that the Fourier transform of the autocorrelation function of a stationary process is equal to the power spectral density (PSD) of that process. For spatial data, this relationship allows us to compute the PSD of the velocity field from the spatial autocorrelation function, and vice versa. The theorem for spatial data is expressed as:

$$
S(\mathbf{k}) = \mathcal{F}\{ R(\Delta x, \Delta y) \}
$$

where $S(\mathbf{k})$ represents the spatial power spectral density as a function of the spatial frequency vector $\mathbf{k}$, and $\mathcal{F}$ denotes the Fourier transform.

Conversely, we can obtain the spatial autocorrelation function from the PSD using the inverse Fourier transform:

$$
R(\Delta x, \Delta y) = \mathcal{F}^{-1}\{ S(\mathbf{k}) \}
$$

### Computing the Spatial Autocorrelation Function:

To compute the spatial autocorrelation function for a single frame of PIV data:

1. **Calculate the Velocity Magnitude**: First, compute the magnitude of the velocity vector at each point in the 2D space.

$$
\text{magnitude} = \sqrt{u^2 + v^2}
$$

where $u$ and $v$ are the $x$ and $y$ components of the velocity vector, respectively.

2. **Compute the 2D Fourier Transform**: Then, apply a 2D Fourier transform to the magnitude of the velocity vectors to move from the spatial domain to the frequency domain.

$$
F(\mathbf{k}) = \mathcal{F}\{ \text{magnitude} \}
$$

3. **Calculate the Power Spectral Density**: Multiply the Fourier-transformed velocity magnitude by its complex conjugate to get the power spectral density.

$$
S(\mathbf{k}) = F(\mathbf{k}) \cdot F^*(\mathbf{k})
$$

4. **Inverse Fourier Transform**: Finally, apply the inverse 2D Fourier transform to the PSD to obtain the spatial autocorrelation function.

$$
R(\Delta x, \Delta y) = \mathcal{F}^{-1}\{ S(\mathbf{k}) \}
$$

This spatial autocorrelation function will describe how velocity magnitudes are correlated with themselves over spatial shifts in the PIV data. It helps to identify patterns and coherent structures within the flow. For example, here we can visualize the spatial autocorrelation of the entire first frame.

In [ ]:
v = df.pivot(index='y [um]', columns='x [um]', values="magnitude [um/s]").values

# Calculate the autocorrelation function with fourier transform
full_product = np.fft.fft2(v) * np.conj(np.fft.fft2(v))

# get the real part of the inverse fourier transform (autocorrlation function)
inverse = np.real(np.fft.ifft2(full_product))

# normalize the autocorrelation function
normalized_inverse = inverse / inverse[0, 0]

# shift the autocorrelation function so that the center is at the center of the image
shifted_inverse =  np.fft.fftshift(normalized_inverse)

# plot inverse fourier transform of the product of the fourier transform of the magnitude of the velocity
plt.figure(figsize=(10, 6))  # Set the figure size
im = plt.imshow(shifted_inverse, cmap='viridis', origin='lower', extent=[-2762/2, 2762/2, -2762/2, 2762/2])
plt.xlabel('x [um]')
plt.ylabel('y [um]')
cbar = plt.colorbar(im)
cbar.set_label('Autocorrelation')
plt.title('Autocorrelation')
plt.show()


Here we compute the spatial autocorrelation with a space lag $r$ as follows:

$$
\frac{\text{{inverse}}[r, r] + \text{{inverse}}[-r, -r]}{\text{{shape}}[0] \times \text{{shape}}[1]}
$$

Notice that the maximum number of time lags $r$ are 127 because we have that number of points per axis.

In [ ]:
# Define the number of r values
r_values = v.shape[0]//2

# Initialize an array to store the autocorrelation results
results = np.zeros(r_values)

# Compute the autocorrelation for each r value
for r in range(r_values):
    # Compute the autocorrelation value for the current r value
    autocorrelation_value = (inverse[r, r] + inverse[-r, -r]) / (v.shape[0] * v.shape[1])
    # Store the autocorrelation value in the results array
    results[r] = autocorrelation_value

# Normalize the results array
results = results / results[0]

From the spatial correlation function, a correlation length is discerned, representing the extent over which two points retain significant correlation. This length is derived from the velocity autocorrelation, $A_{\text{vel}}(r,\tau)$, fitted to an exponential decay model:
$$
A_{\text{vel}}(r,\tau) = Ae^{-\frac{\tau}{B}} + C.
$$
The correlation length, symbolized by $\lambda(\tau)$, is then given by:
$$
\lambda(\tau) = -B \times \log \left( \frac{0.3-C}{A} \right).
$$
Temporal averages of this length, represented by $\langle \lambda(\tau) \rangle$, furnish an overarching correlation length for the entire experimental dataset.


Now, remember that the PIVlab output is a 127x127 grid of vectors, each representing a chunk of distance. How much?

In [ ]:
intervector_distance_microns = (df["y [m]"].max() - df["y [m]"].min()) * 1000000 / v.shape[0]
intervector_distance_microns

The real distance between vectors, and therefore spatial lags $r$, is 21.75 microns.

In [ ]:
# Define the exponential decay model
def exponential_decay(tau, A, B, C):
    return A * np.exp(-tau/B) + C

# Fit the results to the exponential decay model
params, _ = curve_fit(exponential_decay, np.arange(len(results)), results, maxfev=5000)
A, B, C = params
fitted_values = exponential_decay(np.arange(r_values), A, B, C)

# Compute the correlation length
lambda_tau = -B * np.log((0.3 - C) / A) * intervector_distance_microns
print(f"The correlation length is {lambda_tau}")

In [ ]:
x = np.arange(len(results)) * intervector_distance_microns

# Plot the autocorrelation function
plt.figure(figsize=(12, 6))
plt.plot(x, results, 'o', label='Autocorrelation')
plt.plot(x, fitted_values , label='Exponential fit')
plt.axvline(lambda_tau, color='black', linestyle='--', label=f'Correlation length = {np.round(lambda_tau, 2)}')
plt.xlabel('r (spatial lags)')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation')
plt.legend()
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()

Half a millimiter! Now let's calculate the power (joules per second), given by the formula:

$$
P_{\text{flow}} \approx V_0 \mu \left( \frac{\left< \vec{v}(r,t) \right>}{ L } \right)^2 
$$

Where:
- V0 is volume
- µ is the viscosity of water (1mPa*S)
- $\left< \vec{v}(r,t) \right>$ is mean velocity 
- L is correlation length

In [ ]:
v0 = 2 # µl
µ = 1 # mPa*S

# Calculate power
df["Power (W)"] = v0 * µ * (df["magnitude [m/s]"].mean()/lambda_tau)**2
df

So far we have plotted a single frame of data and analyzed the correlation length to estimate power. Now let's build a function that takes a frame and gives us an updated dataframe w

The goal here is to build a set of functions to analye this and all the future activedrops data. 

In [ ]:
def df_piv(file, volume=2):

    df = pd.read_csv(file, skiprows=2).fillna(0)

    df['x [um]'] = df['x [m]'] * 1E6
    df['y [um]'] = df['y [m]'] * 1E6
    df['u [um/s]'] = df['u [m/s]'] * 1E6
    df['v [um/s]'] = df['v [m/s]'] * 1E6
    df['magnitude [um/s]'] = df['magnitude [m/s]'] * 1E6

    # Obtain square grid of velocity magnitudes
    v = df.pivot(index='y [um]', columns='x [um]', values="magnitude [um/s]").values

    # Calculate the autocorrelation function with fourier transform
    full_product = np.fft.fft2(v) * np.conj(np.fft.fft2(v))

    # get the real part of the inverse fourier transform (autocorrlation function)
    inverse = np.real(np.fft.ifft2(full_product))

    # normalize the autocorrelation function
    normalized_inverse = inverse / inverse[0, 0]

    # Define the number of r values
    r_values = v.shape[0]//2

    # Initialize an array to store the autocorrelation results
    results = np.zeros(r_values)

    # Compute the autocorrelation for each r value
    for r in range(r_values):
        # Compute the autocorrelation value for the current r value
        autocorrelation_value = (inverse[r, r] + inverse[-r, -r]) / (v.shape[0] * v.shape[1])
        # Store the autocorrelation value in the results array
        results[r] = autocorrelation_value

    # Normalize the results array
    results = results / results[0]

    # Define the exponential decay model
    def exponential_decay(tau, A, B, C):
        return A * np.exp(-tau/B) + C

    # Fit the results to the exponential decay model
    params, _ = curve_fit(exponential_decay, np.arange(len(results)), results, maxfev=5000)
    A, B, C = params
    fitted_values = exponential_decay(np.arange(r_values), A, B, C)

    # Obtain the true distance between spatial lags/vectors 
    intervector_distance_microns = (df["y [um]"].max() - df["y [um]"].min()) / v.shape[0]

    # Compute the correlation length (lambda tau)
    lambda_tau = -B * np.log((0.3 - C) / A) * intervector_distance_microns

    # Add correlation length to dataframe
    df["correlation length (µm)"] = lambda_tau

    # Calculate power and add to dataframe. Remember this is with m/s, not microns!
    v0 = volume # µl
    µ = 1 # mPa*S
    df["Power (W)"] = v0 * µ * (df["magnitude [m/s]"].mean()/lambda_tau)**2

    # Add mean velocity
    df["mean velocity [um/s]"] = df["magnitude [um/s]"].mean()

    # Add file name
    df["file name"] = os.path.basename(file).split('.')[0]

    # Simply reorganize
    df = pd.concat([df.iloc[:, 12:], df.iloc[:, 4:12]], axis=1)

    return df

link = "../../data/k401bio-250nM-piv/piv_data/PIVlab_****.txt"
files = sorted(glob.glob(link))
df = df_piv(files[50])
df

In [ ]:
def piv_heatmap(df, feature, vmax=10, output_dir=None):

    vals = df.pivot(index='y [um]', columns='x [um]', values=feature).values

    # Plot the magnitude of the velocity
    plt.figure(figsize=(10, 6))  # Set the figure size
    im = plt.imshow(vals, cmap='viridis', origin='lower', extent=[-2762/2, 2762/2, -2762/2, 2762/2], vmin=0, vmax=vmax)
    plt.xlabel('x [um]')
    plt.ylabel('y [um]')
    cbar = plt.colorbar(im)
    cbar.set_label(feature)
    plt.title(f'ActiveDROPS PIV - {df["file name"][0]}')
    
    if output_dir:
        plt.savefig(output_dir, format='jpg', dpi=250)
        plt.close()
    else:
        plt.show()

output_dir = "../../data/k401bio-250nM-piv/plots/heatmap.jpg"
piv_heatmap(df, "magnitude [um/s]", vmax=10, output_dir=output_dir)

Great! Now we have functions to process and plot single frames. Now let's go on to write a function that stores every dataframe per movie frame in a list, and generates plots from that.

In [ ]:
# Import plotting utilities
import os
import glob
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
from ipywidgets import interact, FloatSlider
import matplotlib.gridspec as gridspec
from scipy.ndimage import gaussian_filter
import colorcet as cc
import holoviews as hv
import panel as pn
pn.extension()

def df_piv(file, volume=2):
    """
    Processes a PIV data file and computes various parameters.

    Args:
    - file (str): Path to the PIV data file.
    - volume (float): The volume in microliters for power calculation.

    Returns:
    - DataFrame: Processed DataFrame with additional computed columns.
    """

    # Read and preprocess the dataframe
    df = pd.read_csv(file, skiprows=2).fillna(0)
    # Convert measurements to micrometers and micrometers per second
    df['x [um]'] = df['x [m]'] * 1E6
    df['y [um]'] = df['y [m]'] * 1E6
    df['u [um/s]'] = df['u [m/s]'] * 1E6
    df['v [um/s]'] = df['v [m/s]'] * 1E6
    df['magnitude [um/s]'] = df['magnitude [m/s]'] * 1E6

    # Obtain square grid of velocity magnitudes
    v = df.pivot(index='y [um]', columns='x [um]', values="magnitude [um/s]").values

    # Calculate the autocorrelation function with Fourier transform
    full_product = np.fft.fft2(v) * np.conj(np.fft.fft2(v))
    inverse = np.real(np.fft.ifft2(full_product))  # Real part of the inverse Fourier transform
    normalized_inverse = inverse / inverse[0, 0]   # Normalize the autocorrelation function

    # Define the number of r values and initialize an array for the results
    r_values = v.shape[0] // 2
    results = np.zeros(r_values)

    # Compute the autocorrelation for each r value
    for r in range(r_values):
        autocorrelation_value = (inverse[r, r] + inverse[-r, -r]) / (v.shape[0] * v.shape[1])
        results[r] = autocorrelation_value

    # Normalize the results array
    results = results / results[0]

    # Fit the results to an exponential decay model
    def exponential_decay(tau, A, B, C):
        return A * np.exp(-tau / B) + C

    params, _ = curve_fit(exponential_decay, np.arange(len(results)), results, maxfev=5000)
    A, B, C = params
    fitted_values = exponential_decay(np.arange(r_values), A, B, C)

    # Compute correlation length and other parameters
    intervector_distance_microns = (df["y [um]"].max() - df["y [um]"].min()) / v.shape[0]
    lambda_tau = -B * np.log((0.3 - C) / A) * intervector_distance_microns
    df["correlation length (µm)"] = lambda_tau

    v0 = volume  # µl
    µ = 1        # mPa*S
    df["Power (W)"] = v0 * µ * (df["magnitude [m/s]"].mean() / lambda_tau) ** 2
    df["mean velocity [um/s]"] = df["magnitude [um/s]"].mean()
    df["file name"] = os.path.basename(file).split('.')[0]

    # Reorganize DataFrame
    df = pd.concat([df.iloc[:, 12:], df.iloc[:, 4:12]], axis=1)

    return df


def piv_heatmap(df, feature, vmin, vmax, output_dir=None):
    """
    Generates and saves/renders a heatmap of a specified feature from the PIV data.

    Args:
    - df (DataFrame): DataFrame containing PIV data.
    - feature (str): Column name for the feature to plot.
    - vmin (float): Minimum value for colormap scaling.
    - vmax (float): Maximum value for colormap scaling.
    - output_dir (str, optional): Directory to save the plot. If None, the plot is shown.
    """

    vals = df.pivot(index='y [um]', columns='x [um]', values=feature).values

    # Plot setup
    plt.figure(figsize=(10, 6))
    im = plt.imshow(vals, cmap='viridis', origin='lower', extent=[-2762/2, 2762/2, -2762/2, 2762/2], vmin=vmin, vmax=vmax)
    plt.xlabel('x [um]')
    plt.ylabel('y [um]')
    cbar = plt.colorbar(im)
    cbar.set_label(feature)
    plt.title(f'ActiveDROPS PIV - {df["file name"][0]}')
    
    # Save or show the plot
    if output_dir:
        os.makedirs(os.path.dirname(output_dir), exist_ok=True)
        plt.savefig(output_dir, format='jpg', dpi=250)
        plt.close()
    else:
        plt.show()


def process_piv_files(path, max_frame=10):
    """
    Processes multiple PIV files from a given directory.

    Args:
    - path (str): Path pattern to the PIV files.

    Returns:
    - List[DataFrame]: List of processed DataFrames.
    """
    files = sorted(glob.glob(path))
    dataframes = [df_piv(file) for file in files[:max_frame]]
    return dataframes


def generate_heatmaps(dataframes, feature, output_dir_base=None, vmin=None, vmax=None):
    """
    Generates heatmaps for a list of DataFrames and a specified feature.

    Args:
    - dataframes (List[DataFrame]): List of DataFrame objects to plot.
    - feature (str): The feature to plot.
    - output_dir_base (str, optional): Base directory to save heatmaps. If None, heatmaps are displayed.
    - vmin (float, optional): Minimum value for colormap scaling. If None, computed from data.
    - vmax (float, optional): Maximum value for colormap scaling. If None, computed from data.
    """
    # Removing units and special characters from the feature name for folder creation
    feature_name_for_folder = ''.join(filter(str.isalnum, feature.split('[')[0])).strip()

    # Calculate vmin and vmax if not provided
    if vmin is None:
        vmin = min(df[feature].min() for df in dataframes)
    if vmax is None:
        vmax = max(df[feature].max() for df in dataframes)

    for i, df in enumerate(dataframes):
        output_dir = None
        if output_dir_base:
            # Creating a specific folder for the feature
            feature_folder = os.path.join(output_dir_base, feature_name_for_folder)
            os.makedirs(feature_folder, exist_ok=True)
            output_dir = os.path.join(feature_folder, f"heatmap_{i}.jpg")
        piv_heatmap(df, feature, vmin=vmin, vmax=vmax, output_dir=output_dir)

# Example usage
input_dir = "../../data/k401bio-250nM-piv/piv_data/PIVlab_****.txt"
dataframes = process_piv_files(input_dir, max_frame=10)

output_dir = "../../data/k401bio-250nM-piv/plots/"
generate_heatmaps(dataframes, 'simple strain [1/s]', output_dir_base=output_dir) 

In [ ]:
for i in dataframes[0].columns[9:]:
    generate_heatmaps(dataframes, i, output_dir_base=output_dir) 

Amazing! Now we can process all the PIV output! There a couple interesting plots we're missing, namely velocity and power over time. Let's proceed to obtain that.

In [ ]:
def piv_time_series(dataframes, time_interval_min=0.05):

    df_ts = pd.DataFrame(columns=['file name', 'Power (W)', 'mean velocity [um/s]'])

    for df in dataframes:
        file_name = df.loc[0, 'file name']
        power = df.loc[0, 'Power (W)']
        velocity = df.loc[0, 'mean velocity [um/s]']
        # Make sure the column names match those in new_df
        new_row = pd.DataFrame({'file name': [file_name], 'Power (W)': [power], 'mean velocity [um/s]': [velocity]})
        df_ts = pd.concat([df_ts, new_row], ignore_index=True)

    df_ts = df_ts.reset_index(drop=False)
    df_ts = df_ts.rename(columns={'index': 'time (min)'})
    df_ts['time (min)'] = df_ts['time (min)'] * time_interval_min
    return df_ts

df_ts = piv_time_series(dataframes)

In [ ]:
from scipy.ndimage import gaussian_filter


def plot_time_series(df_ts, feature, sigma=0):

    if feature == 'velocity':
        y = gaussian_filter(df_ts['mean velocity [um/s]'], sigma=sigma)
    if feature == 'power':
        y = gaussian_filter(df_ts['Power (W)'], sigma=sigma)

    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.plot(df_ts['time (min)'], y, label='K401-biotin')
    ax.fill_between(df_ts['time (min)'], y, alpha=0.2)

    ax.set_xlabel('time (min)')
    ax.set_ylabel('Velocity (µm/s)')
    ax.legend()

    plt.show()

plot_time_series(df_ts, 'power')

Cool! Now we have a whole set of functions to process our PIV data! Let's make these functions a package called `pivdrops`. Here is the implementation:

#### 1. Import relevant pachages.

The pivdrops package contains most of the imports we need for this pipeline.

In [ ]:
## Import relevant libraries
import os
import glob
import sys
sys.path.append('../../activedrops')

## Including ourselves
import pivdrops
pivdrops.set_plotting_style()

#### 2. Process PIV files 

The function takes an input as follows, which is the output of PIVlab. Each .txt frame in the directory correspont to a movie frame, and our function turns it into an useful pandas Dataframe.

In [ ]:
# Process the PIV files
input_dir = "../../data/k401bio-250nM-piv/piv_data/PIVlab_****.txt"
output_dir = "../../data/k401bio-250nM-piv/plots/"

dataframes = pivdrops.process_piv_files(input_dir, max_frame=None)

dataframes[0].head()

#### 3. Plot vector fields (velocity & derivatives)

Thhis function takes all dataframes and turns the features into movies (created in your output_dir)

In [ ]:
[pivdrops.generate_heatmaps(dataframes, i, output_dir_base=output_dir) for i in dataframes[0].columns[10:]]


#### 4. Generate a single dataframe with velocity and power

We can also multiply these two by time to get distance and work respectively

In [ ]:
df = pivdrops.piv_time_series(dataframes)

### 5. Plot global dynamics

Called global because come from mean values 

In [ ]:
# List of feature names to plot
features_to_plot = ['velocity', 'power', 'distance', 'work']

for feature in features_to_plot:
    pivdrops.plot_time_series(df, feature, output_dir=output_dir, sigma=0.5)


#### Generate a super function!

In [ ]:
from PIL import Image, ImageEnhance, ImageOps  # Added ImageOps here


In [ ]:
import os
import glob
import sys
import pandas as pd
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()
from natsort import natsorted  # Import for natural sorting
import cv2
import os
import re
from PIL import Image, ImageEnhance, ImageOps  # Added ImageOps here


def sorted_alphanumeric(data):
    """
    Helper function to sort data in human-readable alphanumeric order.
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

def make_movies_from_features(base_directory, fps):
    # Find all subdirectories in the base directory
    subdirectories = [d for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]

    for feature in subdirectories:
        feature_directory = os.path.join(base_directory, feature)
        output_filename = os.path.join(base_directory, f"{feature}.avi")

        # Get all the .jpg files from the feature directory
        images = [img for img in os.listdir(feature_directory) if img.endswith(".jpg")]

        # Skip if no images are found
        if not images:
            print(f"No images found in {feature_directory}, skipping movie creation.")
            continue

        # Sort the images in alphanumeric order
        images = sorted_alphanumeric(images)

        # Read the first image to get the width and height
        frame = cv2.imread(os.path.join(feature_directory, images[0]))
        height, width, layers = frame.shape

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

        # Loop through all the images and add them to the video
        for image in images:
            frame = cv2.imread(os.path.join(feature_directory, image))
            out.write(frame)

        # Release everything when the job is finished
        out.release()

def save_dataframes(dataframes, output_dir, prefix='df'):
    """Saves a list of DataFrames to the specified directory."""
    dataframes_dir = os.path.join(output_dir, 'dataframes')
    os.makedirs(dataframes_dir, exist_ok=True)

    for i, df in enumerate(dataframes):
        df_path = os.path.join(dataframes_dir, f'{prefix}_{i}.csv')
        df.to_csv(df_path, index=False)

def load_dataframes(output_dir, prefix='df'):
    """Loads DataFrames from the specified directory."""
    dataframes_dir = os.path.join(output_dir, 'dataframes')
    if not os.path.exists(dataframes_dir):
        return None

    df_files = sorted(glob.glob(os.path.join(dataframes_dir, f'{prefix}_*.csv')))
    if not df_files:
        return None

    return [pd.read_csv(df_file) for df_file in df_files]

def convert_images(input_dir, output_dir, max_frame=None, brightness_factor=1, contrast_factor=1):
    """Converts and adjusts images from input_dir and saves them in output_dir."""
    os.makedirs(output_dir, exist_ok=True)
    input_files = natsorted(glob.glob(os.path.join(input_dir, '*.jpg')))

    if max_frame is not None:
        input_files = input_files[:max_frame]

    output_files = natsorted(glob.glob(os.path.join(output_dir, '*.tif')))

    if len(input_files) == len(output_files):
        print(f"Conversion already completed for {output_dir}. Skipping...")
        return

    num_digits = len(str(len(input_files)))

    for i, file_name in enumerate(input_files):
        image = Image.open(file_name).convert("L")
        image_resized = image.resize((2048, 2048), Image.LANCZOS)

        # Adjust brightness
        enhancer = ImageEnhance.Brightness(image_resized)
        image_brightened = enhancer.enhance(brightness_factor)

        # Adjust contrast
        enhancer = ImageEnhance.Contrast(image_brightened)
        image_contrasted = enhancer.enhance(contrast_factor)

        padded_index = str(i + 1).zfill(num_digits)
        base_file_name = f'converted_image_{padded_index}.tif'
        processed_image_path = os.path.join(output_dir, base_file_name)
        image_contrasted.save(processed_image_path, format='TIFF', compression='tiff_lzw')

def combine_timeseries_dataframes(base_data_dir, conditions, subconditions):
    combined_df = pd.DataFrame()

    for condition in conditions:
        for subcondition in subconditions:
            file_path = os.path.join(base_data_dir, condition, subcondition, 'plots', 'dataframes', 'timeseries_df_0.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                df['Condition'] = f'{condition} {subcondition}'
                combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df


def process_piv_workflow(conditions, subconditions, base_data_dir, max_frame=None, volume_ul=2, brightness_factor=0.8, contrast_factor=0.8):
    """Main workflow to process PIV data."""

    feature_limits = {
        'magnitude [um/s]': (0, 10),
        'vorticity [1/s]': (-0.03, 0.03),
        'divergence [1/s]': (-0.03, 0.03),
        # 'dcev [1]': (0, 250),
        'simple shear [1/s]': (-0.03, 0.03),
        'simple strain [1/s]': (-0.03, 0.03),
        'vector direction [degrees]': (-180, 180),
    }
    
    for condition in conditions:
        for subcondition in subconditions:
            # Define directories
            input_image_dir = os.path.join(base_data_dir, condition, subcondition, "piv_movie")
            output_image_dir = os.path.join(base_data_dir, condition, subcondition, "piv_movie_8bit_2048x2048")
            output_dir = os.path.join(base_data_dir, condition, subcondition, "plots")

            # Convert images if not already done
            convert_images(input_image_dir, output_image_dir, max_frame, brightness_factor, contrast_factor)

            # Process PIV files if not already done
            dataframes_path = os.path.join(output_dir, 'dataframes', 'df_0.csv')
            if not os.path.exists(dataframes_path):
                input_dir = os.path.join(base_data_dir, condition, subcondition, "piv_data", "PIVlab_****.txt")
                dataframes = pivdrops.process_piv_files(input_dir, volume=volume_ul, max_frame=max_frame)
                # save_dataframes(dataframes, output_dir)
            else:
                dataframes = [pd.read_csv(dataframes_path)]

            # Process time series data if not already done
            timeseries_df_path = os.path.join(output_dir, 'dataframes', 'timeseries_df_0.csv')
            if not os.path.exists(timeseries_df_path):
                df = pivdrops.piv_time_series(dataframes, time_interval_seconds=3)
                save_dataframes([df], output_dir, prefix='timeseries_df')
            else:
                df = pd.read_csv(timeseries_df_path)

            # Plot time series for each feature
            combined_df = combine_timeseries_dataframes(base_data_dir, conditions, subconditions)

            # Plot combined time series data
            for feature in ['velocity', 'power', 'distance', 'work']:
                pivdrops.plot_combined_time_series(combined_df, feature, sigma=1, output_dir=output_dir)  # Specify output_dir if needed

            # Generate heatmaps for each feature
            for feature, (vmin, vmax) in feature_limits.items():
                pivdrops.generate_heatmaps(dataframes, feature, vmin=vmin, vmax=vmax, output_dir_base=output_dir, image_path=output_image_dir)

            # Make movies
            make_movies_from_features(output_dir, fps=120)

                

# Example usage
conditions = ['k401']
subconditions = ['rep1']
base_data_dir = "../../data/02-ActiveDROPSfig2-K401_Kif3/"
process_piv_workflow(conditions, subconditions, base_data_dir, max_frame=None, volume_ul=2)


In [ ]:
import sys
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()

velocity_limits = (0, 10)
other_limits = (-0.01, 0.01)

# velocity_limits = (0, None)
# other_limits = (None, None)

feature_limits = {
    'magnitude [um/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    'dcev [1]': (0, 250),
    'simple shear [1/s]': other_limits,
    'simple strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}

# Example usage
conditions = ['250nM']
subconditions = ['2ul']
data_path = "../../data/01-k401-biotin_strep/"
pivdrops.process_piv_data(data_path, max_frame=None, conditions=conditions, subconditions=subconditions, time_interval_seconds=3, feature_limits=feature_limits, frame_rate=120)
pivdrops.plot_combined_timeseries(conditions, subconditions, data_path)
